In [1]:
import os, glob
import tensorflow as tf
import cv2
import numpy as np

In [2]:
import matplotlib.pyplot as plt 
%matplotlib inline

In [3]:
dataset_dir='/home/admin1/Programming/HIWI_MRT/gp-gan_augmentation/GP-GAN/DataBase/train_data'

out_path_train='/home/admin1/Programming/HIWI_MRT/gp-gan_augmentation/GP-GAN/DataBase/train.tfrecords'
out_path_val='/home/admin1/Programming/HIWI_MRT/gp-gan_augmentation/GP-GAN/DataBase/val.tfrecords'

In [4]:
def _int64_feature(value):
    """
    Wrapper for inserting int64 features into Example proto.
    :param value:
    :return:
    """
    if not isinstance(value, list):
        value = [value]
    value_tmp = []
    is_int = True
    for val in value:
        if not isinstance(val, int):
            is_int = False
            value_tmp.append(int(float(val)))
    if not is_int:
        value = value_tmp
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

In [5]:
def _float_feature(value):
    """
    Wrapper for inserting float features into Example proto.
    :param value:
    :return:
    """
    if not isinstance(value, list):
        value = [value]
    value_tmp = []
    is_float = True
    for val in value:
        if not isinstance(val, int):
            is_float = False
            value_tmp.append(float(val))
    if is_float is False:
        value = value_tmp
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

In [6]:
def _bytes_feature(value):
    """
    Wrapper for inserting bytes features into Example proto.
    :param value:
    :return:
    """
    if not isinstance(value, bytes):
        if not isinstance(value, list):
            value = value.encode('utf-8')
        else:
            value = [val.encode('utf-8') for val in value]
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

In [7]:
def create_tf_example(encoded_image_data_cp, encoded_image_data_bg):

    image_format = b'jpg'

    height, width, _ = encoded_image_data_cp.shape
    encoded_image_data_cp = encoded_image_data_cp.tostring()
    encoded_image_data_bg = encoded_image_data_bg.tostring()

    feature_dict = {
        'image/height': _int64_feature(height),
        'image/width': _int64_feature(width),
        'image/encoded_cp': _bytes_feature(encoded_image_data_cp),
        'image/encoded_bg': _bytes_feature(encoded_image_data_bg),
        'image/format': _bytes_feature(image_format)
    }

    tf_example = tf.train.Example(features=tf.train.Features(feature=feature_dict))

    return tf_example

In [8]:
def create_copy_pastes(src_paths, writer, process_name):
    dataset_size = len(src_paths)
    id_data = 1
    print(" %s start " % process_name)
    for src_path in src_paths:
        src_name = src_path.split('/')[-1]
        dst_path = os.path.join(dataset_dir,'dst',src_name)
        
        src = cv2.cvtColor(cv2.imread(src_path), cv2.COLOR_BGR2RGB)  
        dst = cv2.cvtColor(cv2.imread(dst_path), cv2.COLOR_BGR2RGB)  

        tf_example = create_tf_example(src, dst)
        writer.write(tf_example.SerializeToString())
        
        print("------------------- %s rest --------------------" % (dataset_size - id_data))
        id_data += 1

In [9]:
def main():
    val_ratio = 0.2

    writer_train = tf.python_io.TFRecordWriter(out_path_train)
    writer_val = tf.python_io.TFRecordWriter(out_path_val)

    src_paths = glob.glob(os.path.join(dataset_dir,'src','*.jpg'))
    
    val_end = int(val_ratio * len(src_paths))

    create_copy_pastes(src_paths[val_end:], writer_train, 'Train')
    writer_train.close()

    create_copy_pastes(src_paths[:val_end], writer_val, 'Val')
    writer_val.close()

In [10]:
if __name__ == '__main__':
    main()

 Train start 
------------------- 24 rest --------------------
------------------- 23 rest --------------------
------------------- 22 rest --------------------
------------------- 21 rest --------------------
------------------- 20 rest --------------------
------------------- 19 rest --------------------
------------------- 18 rest --------------------
------------------- 17 rest --------------------
------------------- 16 rest --------------------
------------------- 15 rest --------------------
------------------- 14 rest --------------------
------------------- 13 rest --------------------
------------------- 12 rest --------------------
------------------- 11 rest --------------------
------------------- 10 rest --------------------
------------------- 9 rest --------------------
------------------- 8 rest --------------------
------------------- 7 rest --------------------
------------------- 6 rest --------------------
------------------- 5 rest --------------------
-----------

/home/admin1/Programming/Django/env_dj/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  
/home/admin1/Programming/Django/env_dj/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  import sys
